## ETL , Extract, Transform, Load

| pacakge | use   | DataType   |
|------|------|------|
|   mllib  | RDD API| pyspark.mllib.linalg.Vector / pyspark.mllib.linalg.Matrix|
|   ml  | DataFrame API| pyspark.ml.linalg.Vector /pyspark.ml.linalg.Matrix|

### RDD 변환

| 구분 | descript   |
|------|------|
|   Vector  | numpy vector와 같은 기능을 한다. dense와 sparse vector로 구분한다.|
|   Labeled Point  | 분류를 의미하는 클래스 또는 label과 속성 features 이 묶인 구조로서, 지도학습 supervised learning을 할 경우 사용된다.|
|   Matrix  | 	numpy matrix와 같은 특징을 가진다.|


#### Vectors
- dense vector : 모든 행렬이 값을 가지고 있다.
- sparse vector : 빈 값이 많아서, 값이 있는 경우 그 값이 있는 인덱스로 표현해 배열을 축약. sparse는 실제 값이 없는 요소, '0'을 제거하여 만든 vector

In [2]:
from pyspark.mllib.linalg import Vectors

dv1mllib=Vectors.dense([1.0, 2.1, 3])
print ("Dense vector: {}\nType: {}".format(dv1mllib, type(dv1mllib)))

Dense vector: [1.0,2.1,3.0]
Type: <class 'pyspark.mllib.linalg.DenseVector'>


In [3]:
from pyspark.ml.linalg import Vectors

dv1ml=Vectors.dense([1.0, 2.1, 3])
print ("ml의 dense vector: {}".format(dv1ml))

ml의 dense vector: [1.0,2.1,3.0]


In [4]:
#sparse vector
sv1 = Vectors.sparse(5,[0,1,4],[160.0,69.0,24.0])
type(sv1)

pyspark.ml.linalg.SparseVector

In [8]:
#matrix to sparse vector 
import numpy as np
import scipy.sparse as sps

row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])

mtx = sps.csc_matrix((data, (row, col)), shape=(3, 3))
print(mtx.todense())

[[1 0 2]
 [0 0 3]
 [4 5 6]]


In [10]:
from pyspark.mllib.linalg import Matrices
#(6,4)

dm = Matrices.dense(6,4,[1, 2, 0, 0, 0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 5, 6, 7, 0, 0, 0, 0, 0, 0, 8])
dm.toArray()

array([[1., 0., 0., 0.],
       [2., 3., 0., 0.],
       [0., 0., 5., 0.],
       [0., 4., 6., 0.],
       [0., 0., 7., 0.],
       [0., 0., 0., 8.]])

In [11]:
dm.toSparse()
#열로 개수세서 누적 합, 행으로 행의 위치

SparseMatrix(6, 4, [0, 2, 4, 7, 8], [0, 1, 1, 3, 2, 3, 4, 5], [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], False)

#### Labeled Point

S.4.4 Labeled Point
Labeled point는 로컬벡터로 레이블을 가지고 있는 밀집 또는 희소 행렬을 말한다. 레이블이 있으므로, supervised learning에 요구되는 형식이다. double 형식으로 저장되어 있어야한다. 

|구분| 지도학습을 하기 위한 label과 features의 구성  | 
|------|------|
|   label  | 	supervised learning에서 '구분 값'으로 사용한다. 데이터타입은 'DoubleType'으로 설정되어야 한다.|
|   features  | 	sparse, dense 모두 사용할 수 있다.| 

In [4]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

#### TF(Term Frequency)
단어 빈도를 계산하기 위해 HasingTF를 사용.

In [14]:
# Rdd생성, 
import os
wikiRdd3 = spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))\
    .map(lambda line: line.split())

In [15]:
from pyspark.mllib.feature import HashingTF

hashingTF = HashingTF()
tf = hashingTF.transform(wikiRdd3)
tf.collect() #희소 행렬로, row vector

[SparseVector(1048576, {1026674: 1.0}),
 SparseVector(1048576, {148618: 1.0, 183975: 1.0, 216207: 1.0, 261052: 1.0, 617454: 1.0, 696349: 1.0, 721336: 1.0, 816618: 1.0, 897662: 1.0}),
 SparseVector(1048576, {60386: 1.0, 177421: 1.0, 568609: 1.0, 569458: 1.0, 847171: 1.0, 850510: 1.0, 1040679: 1.0}),
 SparseVector(1048576, {261052: 4.0, 816618: 4.0}),
 SparseVector(1048576, {60386: 4.0, 594754: 4.0}),
 SparseVector(1048576, {21980: 1.0, 70882: 1.0, 274690: 1.0, 357784: 1.0, 549790: 1.0, 597434: 1.0, 804583: 1.0, 829803: 1.0, 935701: 1.0}),
 SparseVector(1048576, {154253: 1.0, 261052: 1.0, 438276: 1.0, 460085: 1.0, 585459: 1.0, 664288: 1.0, 816618: 1.0, 935701: 2.0, 948143: 1.0, 1017889: 1.0}),
 SparseVector(1048576, {270017: 1.0, 472985: 1.0, 511771: 1.0, 718483: 1.0, 820917: 1.0}),
 SparseVector(1048576, {34116: 1.0, 87407: 1.0, 276491: 1.0, 348943: 1.0, 482882: 1.0, 549350: 1.0, 721336: 1.0, 816618: 1.0, 1025622: 1.0}),
 SparseVector(1048576, {1769: 1.0, 151357: 1.0, 500659: 1.0, 54776

In [19]:
from pyspark.mllib.feature import HashingTF, IDF

idf = IDF().fit(tf)
tfidf = idf.transform(tf)
tfidf.collect() #결과는 같지만, tf가 아니라 idf가 들어감.

[SparseVector(1048576, {1026674: 1.7047}),
 SparseVector(1048576, {148618: 1.7047, 183975: 1.7047, 216207: 1.7047, 261052: 1.0116, 617454: 1.7047, 696349: 1.7047, 721336: 1.2993, 816618: 0.7885, 897662: 1.7047}),
 SparseVector(1048576, {60386: 1.2993, 177421: 1.7047, 568609: 1.7047, 569458: 1.7047, 847171: 1.7047, 850510: 1.7047, 1040679: 1.7047}),
 SparseVector(1048576, {261052: 4.0464, 816618: 3.1538}),
 SparseVector(1048576, {60386: 5.1971, 594754: 6.819}),
 SparseVector(1048576, {21980: 1.7047, 70882: 1.7047, 274690: 1.7047, 357784: 1.7047, 549790: 1.7047, 597434: 1.7047, 804583: 1.7047, 829803: 1.7047, 935701: 1.2993}),
 SparseVector(1048576, {154253: 1.7047, 261052: 1.0116, 438276: 1.7047, 460085: 1.7047, 585459: 1.7047, 664288: 1.7047, 816618: 0.7885, 935701: 2.5986, 948143: 1.7047, 1017889: 1.7047}),
 SparseVector(1048576, {270017: 1.7047, 472985: 1.7047, 511771: 1.7047, 718483: 1.7047, 820917: 1.7047}),
 SparseVector(1048576, {34116: 1.7047, 87407: 1.7047, 276491: 1.7047, 3489

### StandardScaler
데이터를 표준화하려면 1) 평균과 표준편차를 계산하고, 2) 측정값에서 평균을 빼고, 표준편차로 나누어 주면 된다. 즉 zscore를 계산하는 것과 같다.

$$ z = \frac {\bar{x_n} - \mu} {\sigma / \sqrt{n}} $$

- from pyspark.mllib.feature import StandardScaler (Rdd)
scaler2 = StandardScaler(withMean=True, withStd=True).fit(_tRdd)

### DataFrame 변환
기계학습에 넘겨줄 입력데이터를 형식에 맞추어야 함.
특징 추출하여 feature vectors를 구성, 지도 학습에서는 class 또는 label 필요\
Labeled Point를 label, features 컬럼으로 분해
RDD LabeledPoint는 label과 vectors로 구성되어 있다

In [21]:
#label, features를 갖고 있는 데이터
#LabeledPoint는 RDD에서 사용하는 구조로서 mllib 라이브러리를 사용
#(DataFrame은 LabeledPoint를 컬럼으로 가지고 있지 않는다.)

from pyspark.mllib.regression import LabeledPoint
p = [LabeledPoint(1, [1.0,2.0,3.0]),
     LabeledPoint(1, [1.1,2.1,3.1]),
     LabeledPoint(0, [1.2,2.2,3.3])]

In [22]:
trainDf=spark.createDataFrame(p)
trainDf.collect()

[Row(features=DenseVector([1.0, 2.0, 3.0]), label=1.0),
 Row(features=DenseVector([1.1, 2.1, 3.1]), label=1.0),
 Row(features=DenseVector([1.2, 2.2, 3.3]), label=0.0)]

mllib labeledPoing을 거치지 않으면, 컬럼명이 _1,_2처럼 출력\
[Row(_1=1, _2=[1.0, 2.0, 3.0]),\
 Row(_1=1, _2=[1.1, 2.1, 3.1]),\
 Row(_1=0, _2=[1.2, 2.2, 3.3])]

In [23]:
from pyspark.ml.linalg import SparseVector # ml ok

_rdd = spark.sparkContext.parallelize([
    (0.0, SparseVector(4, {1: 1.0, 3: 5.5})),
    (1.0, SparseVector(4, {0: -1.0, 2: 0.5}))])
_df=_rdd.toDF()
_df.printSchema()

root
 |-- _1: double (nullable = true)
 |-- _2: vector (nullable = true)



In [24]:
#직접 컬럼명 변경 -> 기계학습
_df=_df.withColumnRenamed('_1', 'label').withColumnRenamed('_2', 'features')
_df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0| (4,[1,3],[1.0,5.5])|
|  1.0|(4,[0,2],[-1.0,0.5])|
+-----+--------------------+



#### 단어빈도
텍스트는 정량데이터가 아니기 때문에, 단어의 빈도에 따라 정량화하여 과학적인 분석을 하게 된다.

텍스트 변환 단계
텍스트를 변환하는 단계를 보자. 순서는 변경될 수 있다.

-단계 1: 단어로 분할 Tokenization
그, 영화는, 매우, 강렬했다, 그냥, 좋았다, 영화관에서, 보는, 동안, 긴장을, 늦출, 수, 없었다, 갑돌이가, 분장한, 악당의, 케릭터가, 만들어지는, 과정은, 흥미롭지, 않을, 수가, 없었다, 무비의, 이야기, 전개는, 빠르고, 무엇이, 진실이고, 거짓인지, 판단할, 수, 없었다, 누가, 이런, 영화를, 좋아, 하지, 않을, 수가, 있겠는가, 이모티콘

-단계 2: 정리\
불필요, 오타 등

-단계 3: 불용어 stopwords 제거\
그, 수, 수가, 수, 이런, 하지, 수가 등

-단계 4: 어간 추출 stemming 영화는, 영화의는 다른 단어지만 조사를 제거하면 동일한 단어\
좋았다, 좋아 단어들은 어근을 판별하면 동일한 단어이다.
영화, 무비의 단어는 이음동의

-단계 5: 계량화
word vector로 만든다.
있다-없다, 단어빈도, TF-IDF 사용할 수 있다.
dense, sparse 모두 가능하다. [1,1,1,1,1,0,0],[0,1,0,1,1,1,1]

### Tokenizer
- corpus는 어떤 주제에 대해 쓰여지거나, 어떤 사람이 작성한 전체 '말뭉치'를 말한다. 여러 문장으로 구성된 텍스트 집합을 말한다.
- document는 문장으로 구성된 문서를 말하지만, 한 문장으로만 구성될 수도, 여러 문장으로 만들어질 수도 있다. 예를 들어, "why she had to go" 같은 한 문장도 document라고 하고, "why she had to go.. I don't know" 역시 마찬가지이다.
- vocabularay는 중복없는 단어 집합을 말하며, 예를 들면, "why","she","had","to","go","where","have" 등은 단어이다.\
Tokenizer는 document를 단어로 분리한다. 분리하는 기준은 whitespace로 공백, TAB, CR, New Line 등이 해당된다.

In [5]:
doc2d=[
    ["When I find myself in times of trouble"],
    ["Mother Mary comes to me"],
    ["Speaking words of wisdom, let it be"],
    ["And in my hour of darkness"],
    ["She is standing right in front of me"],
    ["Speaking words of wisdom, let it be"],
    [u"우리 Let it be"],
    [u"나 Let it be"],
    [u"너 Let it be"],
    ["Let it be"],
    ["Whisper words of wisdom, let it be"]
]

In [6]:
myDf=spark.createDataFrame(doc2d, ['sent'])
myDf.show(truncate = True) #truncate=true, 자르지 않고 모두 출력

+--------------------+
|                sent|
+--------------------+
|When I find mysel...|
|Mother Mary comes...|
|Speaking words of...|
|And in my hour of...|
|She is standing r...|
|Speaking words of...|
|      우리 Let it be|
|        나 Let it be|
|        너 Let it be|
|           Let it be|
|Whisper words of ...|
+--------------------+



In [7]:
from pyspark.ml.feature import Tokenizer
tokenizer= Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(myDf)
tokDf.show()

+--------------------+--------------------+
|                sent|               words|
+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|
|Mother Mary comes...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|
|She is standing r...|[she, is, standin...|
|Speaking words of...|[speaking, words,...|
|      우리 Let it be| [우리, let, it, be]|
|        나 Let it be|   [나, let, it, be]|
|        너 Let it be|   [너, let, it, be]|
|           Let it be|       [let, it, be]|
|Whisper words of ...|[whisper, words, ...|
+--------------------+--------------------+



### RegTokenizer
Tokenizer는 white space로 분리하지만, RegexTokenizer는 단어를 분리하기 위해 정규표현식을 적용할 수 있다. 정규표현식을 사용하여 분리하거나 특정 패턴을 추출할 수 있다. 공백으로 분리할 경우 간단히 정규표현식 \s 패턴을 적용할 수 있다. 한글에는 \w 패턴이 적용되지 않는다.


In [29]:
from pyspark.ml.feature import RegexTokenizer

re= RegexTokenizer(inputCol="sent", outputCol="wordsReg", pattern="\\s+")
#\\s+ 공백이 있을 때 분리하라

In [31]:
reDf=re.transform(myDf)
reDf.show()

+--------------------+--------------------+
|                sent|            wordsReg|
+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|
|Mother Mary comes...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|
|She is standing r...|[she, is, standin...|
|Speaking words of...|[speaking, words,...|
|      우리 Let it be| [우리, let, it, be]|
|        나 Let it be|   [나, let, it, be]|
|        너 Let it be|   [너, let, it, be]|
|           Let it be|       [let, it, be]|
|Whisper words of ...|[whisper, words, ...|
+--------------------+--------------------+



### StopWords 
의미 없는 불필요한 단어들, 

In [34]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="wordsReg", outputCol="nostops")

#how to get stopwords
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

#add my stopWords
_mystopwords=[u"나",u"너", u"우리"]
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

StopWordsRemover_7ee8c47fda11

In [35]:
stopDf=stop.transform(reDf)
stopDf.show()

+--------------------+--------------------+--------------------+
|                sent|            wordsReg|             nostops|
+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[find, times, tro...|
|Mother Mary comes...|[mother, mary, co...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|    [hour, darkness]|
|She is standing r...|[she, is, standin...|[standing, right,...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|      우리 Let it be| [우리, let, it, be]|               [let]|
|        나 Let it be|   [나, let, it, be]|               [let]|
|        너 Let it be|   [너, let, it, be]|               [let]|
|           Let it be|       [let, it, be]|               [let]|
|Whisper words of ...|[whisper, words, ...|[whisper, words, ...|
+--------------------+--------------------+--------------------+



#### CountVectorizer
CountVectorizer는 단어의 빈도 수를 계산한다.\
hasingTF와 같은 결과

In [36]:
#CountVectorizer 는 1차원 입력만 가능함.
#2차원을 1차원으로 만들어주는 함수
from functools import reduce 
doc = reduce(lambda x,y: x+y, doc2d)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')

In [38]:
print (vectorizer.fit_transform(doc))

  (0, 9)	1
  (0, 10)	1
  (1, 5)	1
  (1, 4)	1
  (1, 0)	1
  (2, 7)	1
  (2, 13)	1
  (2, 12)	1
  (2, 3)	1
  (3, 2)	1
  (3, 1)	1
  (4, 8)	1
  (4, 6)	1
  (5, 7)	1
  (5, 13)	1
  (5, 12)	1
  (5, 3)	1
  (6, 3)	1
  (6, 14)	1
  (7, 3)	1
  (8, 3)	1
  (9, 3)	1
  (10, 13)	1
  (10, 12)	1
  (10, 3)	1
  (10, 11)	1


In [39]:
vectorizer.vocabulary_

{'times': 9,
 'trouble': 10,
 'mother': 5,
 'mary': 4,
 'comes': 0,
 'speaking': 7,
 'words': 13,
 'wisdom': 12,
 'let': 3,
 'hour': 2,
 'darkness': 1,
 'standing': 8,
 'right': 6,
 '우리': 14,
 'whisper': 11}

In [ ]:
#dense vector 로 출력
vectorizer.fit_transform(doc).todense()

#### spark CountVectorizer

- minDf는 너무 적게 발생하는 경우 무시, 예를 들어 0.5는 전체 문서의 50%보다 적게 발생하는 단어는 무시, 1.0은 기본 값이고, 100%보다 적게 발생하는 경우 무시하게 된다. 즉, minDf=1.0은 문서 1개 이하에서 나타난 단어는 무시하라는 의미이다. 즉 어떤 단어도 무시하지 말라는 의미이다.
- maxDf는 너무 많이 발생하는 경우 무시, 예를 들어 0.5는 전체 문서의 50%보다 많이 발생하는 경우 무시, 1.0은 100%보다 많이 발생하는 경우 무시 (즉, 어떤 단어도 무시하지 말라는 의미). min_df와 마찬가지로 1.0이 기본 값이다.

In [40]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="nostops", outputCol="cv", vocabSize=30, minDF=1.0)

cvModel = cv.fit(stopDf)
cvDf = cvModel.transform(stopDf)
cvDf.show(3)

#일부 컬럼만 선정해서 출력
#cvDf.select('sent','nostops','cv').show()

+--------------------+--------------------+--------------------+--------------------+
|                sent|            wordsReg|             nostops|                  cv|
+--------------------+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[find, times, tro...|(16,[5,6,8],[1.0,...|
|Mother Mary comes...|[mother, mary, co...|[mother, mary, co...|(16,[10,13,14],[1...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|(16,[0,1,2,3],[1....|
+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



##### TF-IDF 계산
IDF는 자주 나타나는 단어에 대한 가중치를 줄이고, 드물게 나타나는 단어에 가중치를 높이는 방식으로 계산된다.

| 항목 | 설명   | example   |
|------|------|------|
|   tf(d,f)  | 단어 t가 문서 d에서 나타나는 단어의 빈도 수, term frequency| $f_{t,d}$ / (number of words in d) = 1/4 = 0.25
(3번째 문서에 stopwords를 제외하면 4개의 단어, wisdom은 1회 나타난다.)|
|   df  |document frequency 단어가 나타난 문서 수| 3 (wisdom이 포함된 문서는 3)|
|N|number of documents 전체 문서의 수|11 (전체의 문서는 11개)|
|idf|inverse document frequency 단어가 나타난 문서의 비율을 거꾸로|ln(N+1 / df+1) + 1 = log(12/4) + 1 = 1.09861 + 1,, 0으로 나뉘는 것을 방지하기 위해 smoothing, 즉 1을 더한다.|


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
#max_df = 1.0 -> 어떤 단어도 무시하지 말라는 의미
vectorizer = TfidfVectorizer(max_df=1.0, stop_words='english',norm = None)

In [42]:
print(vectorizer.fit_transform(doc))

  (0, 10)	2.791759469228055
  (0, 9)	2.791759469228055
  (1, 0)	2.791759469228055
  (1, 4)	2.791759469228055
  (1, 5)	2.791759469228055
  (2, 3)	1.4054651081081644
  (2, 12)	2.09861228866811
  (2, 13)	2.09861228866811
  (2, 7)	2.386294361119891
  (3, 1)	2.791759469228055
  (3, 2)	2.791759469228055
  (4, 6)	2.791759469228055
  (4, 8)	2.791759469228055
  (5, 3)	1.4054651081081644
  (5, 12)	2.09861228866811
  (5, 13)	2.09861228866811
  (5, 7)	2.386294361119891
  (6, 14)	2.791759469228055
  (6, 3)	1.4054651081081644
  (7, 3)	1.4054651081081644
  (8, 3)	1.4054651081081644
  (9, 3)	1.4054651081081644
  (10, 11)	2.791759469228055
  (10, 3)	1.4054651081081644
  (10, 12)	2.09861228866811
  (10, 13)	2.09861228866811


#### Spark를 사용한 TF-IDF
HashingTF에서의 numFeatures는 $2^n$으로 결정함.
기본은 $2^{18}=262,144$이다. 너무 적게 설정되면 인덱스가 부족하거나 적절하게 매핑될 수 있으니 주의해야 한다.


In [43]:
from pyspark.ml.feature import HashingTF, IDF

# hashTF = HashingTF(inputCol="nostops", outputCol="hash", numFeatures=32) #  mapping indices insufficient
hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf) #hashingTF는 fit하지 않고 transform()

#앞서 hashTF의 결과는 벡터튜플이고, 이를 IDF에 입력으로 넣어준다.
idf = IDF(inputCol="hash", outputCol="idf")

idfModel = idf.fit(hashDf)

idfDf = idfModel.transform(hashDf)

In [45]:
idfDf.select('nostops','hash').show()

+--------------------+--------------------+
|             nostops|                hash|
+--------------------+--------------------+
|[find, times, tro...|(262144,[64317,91...|
|[mother, mary, co...|(262144,[24657,63...|
|[speaking, words,...|(262144,[27556,15...|
|    [hour, darkness]|(262144,[74517,98...|
|[standing, right,...|(262144,[84798,21...|
|[speaking, words,...|(262144,[27556,15...|
|               [let]|(262144,[173339],...|
|               [let]|(262144,[173339],...|
|               [let]|(262144,[173339],...|
|               [let]|(262144,[173339],...|
|[whisper, words, ...|(262144,[151864,1...|
+--------------------+--------------------+



#### Word 2 Vec
벡터('king') - 벡터('man') + 벡터('woman') = 벡터('queen) 이런 연산이 가능해진다. 즉 king 단어벡터에서 man 단어백터를 빼고, woman 단어백터를 더하면, queen 단어백터를 구할 수 있다는 의미이다

In [8]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=3, minCount= 0, inputCol="words", outputCol = "w2v")
model = word2Vec.fit(tokDf)
w2vDf = model.transform(tokDf)

In [9]:
for e in w2vDf.select("w2v").take(3):
    print(e)

Row(w2v=DenseVector([0.0441, -0.008, -0.0792]))
Row(w2v=DenseVector([0.0468, -0.0011, 0.0295]))
Row(w2v=DenseVector([0.0224, 0.0343, -0.0383]))


In [10]:
#숫자 해석 불가, 다만 워드 임베딩해서 얻은 의미 있는 숫자들
model.getVectors().show(truncate=False)

+--------+-----------------------------------------------------------------+
|word    |vector                                                           |
+--------+-----------------------------------------------------------------+
|trouble |[0.11287034302949905,0.1447676420211792,-0.07580796629190445]    |
|mother  |[0.13211971521377563,0.026881175115704536,-0.14194709062576294]  |
|find    |[-0.15181413292884827,-0.004126318264752626,-0.13149575889110565]|
|standing|[0.16076406836509705,0.13818997144699097,-0.09662546962499619]   |
|wisdom, |[0.14846765995025635,0.11940699815750122,-0.018271425738930702]  |
|in      |[0.04129142314195633,0.010938539169728756,-0.13256393373012543]  |
|myself  |[0.08741272240877151,0.07993830740451813,0.0181463323533535]     |
|is      |[0.10587336122989655,-0.03098754584789276,-0.09800274670124054]  |
|darkness|[-0.12206508964300156,-0.1522350162267685,-0.05019150301814079]  |
|우리    |[0.15934604406356812,0.05628468841314316,0.1532307267189026]     |
|

In [14]:
model.findSynonyms("times", 3).show()

+-----+------------------+
| word|        similarity|
+-----+------------------+
|   너|0.9580903053283691|
|   나|0.8260308504104614|
|right|0.8046327829360962|
+-----+------------------+



### N-Gram

In [16]:
from pyspark.ml.feature import NGram

#2-gram -> bigram
ngram = NGram(n=2, inputCol = "words", outputCol="ngrams")
ngramDf = ngram.transform(tokDf)
ngramDf.show()

+--------------------+--------------------+----------------------+
|                sent|               words|                ngrams|
+--------------------+--------------------+----------------------+
|When I find mysel...|[when, i, find, m...|  [when i, i find, ...|
|Mother Mary comes...|[mother, mary, co...|  [mother mary, mar...|
|Speaking words of...|[speaking, words,...|  [speaking words, ...|
|And in my hour of...|[and, in, my, hou...|  [and in, in my, m...|
|She is standing r...|[she, is, standin...|  [she is, is stand...|
|Speaking words of...|[speaking, words,...|  [speaking words, ...|
|      우리 Let it be| [우리, let, it, be]|[우리 let, let it, ...|
|        나 Let it be|   [나, let, it, be]| [나 let, let it, i...|
|        너 Let it be|   [너, let, it, be]| [너 let, let it, i...|
|           Let it be|       [let, it, be]|       [let it, it be]|
|Whisper words of ...|[whisper, words, ...|  [whisper words, w...|
+--------------------+--------------------+----------------------+



In [17]:
for e in ngramDf.select("words","ngrams").take(3):
    print (e)

Row(words=['when', 'i', 'find', 'myself', 'in', 'times', 'of', 'trouble'], ngrams=['when i', 'i find', 'find myself', 'myself in', 'in times', 'times of', 'of trouble'])
Row(words=['mother', 'mary', 'comes', 'to', 'me'], ngrams=['mother mary', 'mary comes', 'comes to', 'to me'])
Row(words=['speaking', 'words', 'of', 'wisdom,', 'let', 'it', 'be'], ngrams=['speaking words', 'words of', 'of wisdom,', 'wisdom, let', 'let it', 'it be'])


#### StringIndexer
문자열 컬럼을 인덱스 컬럼으로 변환\
빈도가 제일 높은 순서로 0.0부터 인덱스 값이 주어진다. 인덱스는 double 형을 가지게 된다. 없는 레이블에 대해서는 예외가 발생할 수 있으므로 (default), setHandleInvalid("skip") 함수로 'skip', 'keep', 'error' 등으로 설정할 수 있다.

| 구분 | descript | example|
|------|------|------|
|   nominal  | 명목 또는 구분 값 cateogry |사자, 호랑이, 사람|
|   ordinal  | 명목값과 다른 점은 순서가 있다.| 키, low,med,high|
|   interval  | 일정한 간격이 있다.|150-165,165-180,180-195|

In [19]:
from pyspark.ml.feature import StringIndexer

labelIndexer = StringIndexer(inputCol="sent", outputCol="sentLabel")
model = labelIndexer.fit(myDf)
siDf=model.transform(myDf)
siDf.show()

+--------------------+---------+
|                sent|sentLabel|
+--------------------+---------+
|When I find mysel...|      5.0|
|Mother Mary comes...|      3.0|
|Speaking words of...|      0.0|
|And in my hour of...|      1.0|
|She is standing r...|      4.0|
|Speaking words of...|      0.0|
|      우리 Let it be|      9.0|
|        나 Let it be|      7.0|
|        너 Let it be|      8.0|
|           Let it be|      2.0|
|Whisper words of ...|      6.0|
+--------------------+---------+



In [21]:
import os
from pyspark.sql.types import *

rdd=spark.sparkContext\
    .textFile(os.path.join('data','ds_spark_heightweight.txt'))
myRdd=rdd.map(lambda line:[float(x) for x in line.split('\t')])

myDf=spark.createDataFrame(myRdd,["id","weight","height"])
myDf.printSchema()

root
 |-- id: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)



In [25]:
from pyspark.ml.feature import Binarizer
binarizer = Binarizer(threshold=68.0, inputCol="weight", outputCol="weight2")

In [26]:
binDf = binarizer.transform(myDf)
binDf.show()
#68보다 크면 1 작으면 0

+----+------+------+-------+
|  id|weight|height|weight2|
+----+------+------+-------+
| 1.0| 65.78|112.99|    0.0|
| 2.0| 71.52|136.49|    1.0|
| 3.0|  69.4|153.03|    1.0|
| 4.0| 68.22|142.34|    1.0|
| 5.0| 67.79| 144.3|    0.0|
| 6.0|  68.7| 123.3|    1.0|
| 7.0|  69.8|141.49|    1.0|
| 8.0| 70.01|136.46|    1.0|
| 9.0|  67.9|112.37|    0.0|
|10.0| 66.78|120.67|    0.0|
|11.0| 66.49|127.45|    0.0|
|12.0| 67.62|114.14|    0.0|
|13.0|  68.3|125.61|    1.0|
|14.0| 67.12|122.46|    0.0|
|15.0| 68.28|116.09|    1.0|
|16.0| 71.09| 140.0|    1.0|
|17.0| 66.46| 129.5|    0.0|
|18.0| 68.65|142.97|    1.0|
|19.0| 71.23| 137.9|    1.0|
|20.0| 67.13|124.04|    0.0|
+----+------+------+-------+
only showing top 20 rows



In [28]:
from pyspark.ml.feature import QuantileDiscretizer

discretizer = QuantileDiscretizer(numBuckets=3, inputCol="height", outputCol="height3")
qdDf = discretizer.fit(binDf).transform(binDf)
qdDf.show(10)
#3등분으로 나눔.

+----+------+------+-------+-------+
|  id|weight|height|weight2|height3|
+----+------+------+-------+-------+
| 1.0| 65.78|112.99|    0.0|    0.0|
| 2.0| 71.52|136.49|    1.0|    1.0|
| 3.0|  69.4|153.03|    1.0|    2.0|
| 4.0| 68.22|142.34|    1.0|    2.0|
| 5.0| 67.79| 144.3|    0.0|    2.0|
| 6.0|  68.7| 123.3|    1.0|    0.0|
| 7.0|  69.8|141.49|    1.0|    2.0|
| 8.0| 70.01|136.46|    1.0|    1.0|
| 9.0|  67.9|112.37|    0.0|    0.0|
|10.0| 66.78|120.67|    0.0|    0.0|
+----+------+------+-------+-------+
only showing top 10 rows



In [29]:
#여러 컬럼을 합쳐서 하나의 feature column으로 만들어 줄 때 사용
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=["weight2","height3"],outputCol="features")

vaDf = va.transform(qdDf)
vaDf.show() #feature column 벡터로 만들어짐

+----+------+------+-------+-------+---------+
|  id|weight|height|weight2|height3| features|
+----+------+------+-------+-------+---------+
| 1.0| 65.78|112.99|    0.0|    0.0|(2,[],[])|
| 2.0| 71.52|136.49|    1.0|    1.0|[1.0,1.0]|
| 3.0|  69.4|153.03|    1.0|    2.0|[1.0,2.0]|
| 4.0| 68.22|142.34|    1.0|    2.0|[1.0,2.0]|
| 5.0| 67.79| 144.3|    0.0|    2.0|[0.0,2.0]|
| 6.0|  68.7| 123.3|    1.0|    0.0|[1.0,0.0]|
| 7.0|  69.8|141.49|    1.0|    2.0|[1.0,2.0]|
| 8.0| 70.01|136.46|    1.0|    1.0|[1.0,1.0]|
| 9.0|  67.9|112.37|    0.0|    0.0|(2,[],[])|
|10.0| 66.78|120.67|    0.0|    0.0|(2,[],[])|
|11.0| 66.49|127.45|    0.0|    1.0|[0.0,1.0]|
|12.0| 67.62|114.14|    0.0|    0.0|(2,[],[])|
|13.0|  68.3|125.61|    1.0|    1.0|[1.0,1.0]|
|14.0| 67.12|122.46|    0.0|    0.0|(2,[],[])|
|15.0| 68.28|116.09|    1.0|    0.0|[1.0,0.0]|
|16.0| 71.09| 140.0|    1.0|    2.0|[1.0,2.0]|
|17.0| 66.46| 129.5|    0.0|    1.0|[0.0,1.0]|
|18.0| 68.65|142.97|    1.0|    2.0|[1.0,2.0]|
|19.0| 71.23|

### pipeline

In [30]:

df = spark.createDataFrame([
        (0, "a b c d e spark", 1.0),
        (1, "b d", 0.0),
        (2, "spark f g h", 1.0),
        (3, "hadoop mapreduce", 0.0),
        (4, "my dog has flea problems. help please.",0.0)
    ], ["id", "text", "label"])

In [31]:
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)

In [32]:
#위에 3개의 작업을 한 번에 함.
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
model = pipeline.fit(df)
myDf = model.transform(df)
myDf.select('label', 'features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262144,[74920,89...|
|  0.0|(262144,[89530,14...|
|  1.0|(262144,[36803,17...|
|  0.0|(262144,[132966,1...|
|  0.0|(262144,[1074,389...|
+-----+--------------------+

